In [26]:
import numpy as np
import functools
import math
from matplotlib import collections as mc
import xml.etree.ElementTree as ET
from collections import namedtuple
import csv
import pprint


In [27]:
# Load the XML file
tree = ET.parse('./data_32.xml')
root = tree.getroot()

# Extract information
dataset = root.find('./info/dataset').text
name = root.find('./info/name').text

Vertex = namedtuple("Vertex", ["id", "isWarehouse", "x", "y"])

Request = namedtuple("Request", ["id", "node_id", "quantity"])

print("Nodes:")
nodes = []
warehouse = None
for node in root.findall('./network/nodes/node'):
    node_id = node.attrib['id']
    node_type = node.attrib['type']
    cx = node.find('cx').text
    cy = node.find('cy').text

    if node_type == '0':
        warehouse = Vertex(node_id, True,  float(cx), float(cy))

    nodes.append((node_id, node_type, cx, cy))

assert warehouse is not None


Nodes:


In [28]:
car_type = None

print("\nFleet:")
for vehicle_profile in root.findall('./fleet/vehicle_profile'):
    departure_node = vehicle_profile.find('departure_node').text
    arrival_node = vehicle_profile.find('arrival_node').text
    capacity = vehicle_profile.find('capacity').text
    car_type = { 'departure_node': departure_node, 'arrival_node': arrival_node, 'capacity': capacity }

requests_list = []

for request in root.findall('./requests/request'):
    request_id = request.attrib['id']
    node = request.attrib['node']
    quantity = request.find('quantity').text
    requests_list.append({'Request ID': request_id, 'Node': node, 'Quantity': quantity})

requests_list







Fleet:


[{'Request ID': '1', 'Node': '2', 'Quantity': '19.0'},
 {'Request ID': '2', 'Node': '3', 'Quantity': '21.0'},
 {'Request ID': '3', 'Node': '4', 'Quantity': '6.0'},
 {'Request ID': '4', 'Node': '5', 'Quantity': '19.0'},
 {'Request ID': '5', 'Node': '6', 'Quantity': '7.0'},
 {'Request ID': '6', 'Node': '7', 'Quantity': '12.0'},
 {'Request ID': '7', 'Node': '8', 'Quantity': '16.0'},
 {'Request ID': '8', 'Node': '9', 'Quantity': '6.0'},
 {'Request ID': '9', 'Node': '10', 'Quantity': '16.0'},
 {'Request ID': '10', 'Node': '11', 'Quantity': '8.0'},
 {'Request ID': '11', 'Node': '12', 'Quantity': '14.0'},
 {'Request ID': '12', 'Node': '13', 'Quantity': '21.0'},
 {'Request ID': '13', 'Node': '14', 'Quantity': '16.0'},
 {'Request ID': '14', 'Node': '15', 'Quantity': '3.0'},
 {'Request ID': '15', 'Node': '16', 'Quantity': '22.0'},
 {'Request ID': '16', 'Node': '17', 'Quantity': '18.0'},
 {'Request ID': '17', 'Node': '18', 'Quantity': '19.0'},
 {'Request ID': '18', 'Node': '19', 'Quantity': '1.0'

In [29]:



vertices = []
requests = []

for node in nodes:
    if node[1] == '0':
        result = True
    else:
        result = False

    vertices.append(Vertex(node[0], result,  float(node[2]), float(node[3])))

for request in requests_list:
    requests.append(Request(request['Request ID'], request['Node'], request['Quantity']))

In [30]:
requests
vertices


[Vertex(id='1', isWarehouse=True, x=82.0, y=76.0),
 Vertex(id='2', isWarehouse=False, x=96.0, y=44.0),
 Vertex(id='3', isWarehouse=False, x=50.0, y=5.0),
 Vertex(id='4', isWarehouse=False, x=49.0, y=8.0),
 Vertex(id='5', isWarehouse=False, x=13.0, y=7.0),
 Vertex(id='6', isWarehouse=False, x=29.0, y=89.0),
 Vertex(id='7', isWarehouse=False, x=58.0, y=30.0),
 Vertex(id='8', isWarehouse=False, x=84.0, y=39.0),
 Vertex(id='9', isWarehouse=False, x=14.0, y=24.0),
 Vertex(id='10', isWarehouse=False, x=2.0, y=39.0),
 Vertex(id='11', isWarehouse=False, x=3.0, y=82.0),
 Vertex(id='12', isWarehouse=False, x=5.0, y=10.0),
 Vertex(id='13', isWarehouse=False, x=98.0, y=52.0),
 Vertex(id='14', isWarehouse=False, x=84.0, y=25.0),
 Vertex(id='15', isWarehouse=False, x=61.0, y=59.0),
 Vertex(id='16', isWarehouse=False, x=1.0, y=65.0),
 Vertex(id='17', isWarehouse=False, x=88.0, y=51.0),
 Vertex(id='18', isWarehouse=False, x=91.0, y=2.0),
 Vertex(id='19', isWarehouse=False, x=19.0, y=32.0),
 Vertex(id=

In [31]:
@functools.lru_cache(maxsize=None)
def distance_function(v1, v2):
    return math.sqrt(((v1.x - v2.x) * (v1.x - v2.x)) + ((v1.y - v2.y) * (v1.y - v2.y)))


def fitness_function(vertices, distance, solution):
    solution_distance = 0
    for x, y in zip(solution, solution[1:] + solution[0:1]):
        solution_distance += distance(vertices[x], vertices[y])
    
    return solution_distance

def initialize_pheromone(N):
    return 0.01 * np.ones(shape=(N,N))


def update_pheromone(pheromones_array, solutions, fitness_values, Q, rho):
    pheromone_update = np.zeros(shape=pheromones_array.shape)
    for solution, fitness_value in zip(solutions, fitness_values):
        for x, y in zip(solution, solution[1:] + solution[0:1]):
            pheromone_update[x, y] += Q / fitness_value
    
    return (1 - rho) * pheromones_array + pheromone_update

def generate_solutions(vertices, requests, pheromones, distance_function, number_of_ants, car_capacity, alpha=1, beta=3):
    
    # Probability of selecting v2 right after v1
    def compute_prob(v1, v2):
        inverse_distance = 1 / distance_function(vertices[v1], vertices[v2])
        tau = pheromones[v1, v2]
        ret = pow(tau, alpha) * pow(inverse_distance, beta)
        return ret if ret > 1e-6 else 1e-6
    
    number_of_vertices = len(vertices)
    for _ in range(number_of_ants):
        available = list(requests)
        solution = [0]  # Start from the warehouse
        capacity = car_capacity
        
        while available:
            probs = np.array([compute_prob(solution[-1], req) for req in available])
            probs /= probs.sum()  # Normalize probabilities
            
            selected_index = np.random.choice(len(available), p=probs)
            selected_request = available[selected_index]
            
            # Check if adding the selected request exceeds the capacity
            if capacity <= 0:
                # Revisit the warehouse if capacity is exhausted
                solution.append(0)
                capacity = car_capacity
            
            if capacity > 0:
                solution.append(selected_request)
                available.remove(selected_request)
                capacity -= 1  # Decrement capacity
        
        # Return to the warehouse after serving all requests
        solution.append(0)
        
        yield solution


def ant_solver(vertices, distance_function, number_of_ants=10, max_iterations=80, alpha=1, beta=3, Q=100, rho=0.8):
    pheromones = initialize_pheromone(len(vertices))
    best_solution = None
    best_fitness = float("inf")
    log_of_best_distances = list()
    
    print("Iteration\tMinimum value\tMean value\tMaximum value")
    
    for i in range(max_iterations):
        candidate_solutions = list(generate_solutions(vertices, requests, pheromones, distance_function, number_of_ants, float(car_type['capacity']), alpha=alpha, beta=beta))
        fitness_values = list(map(lambda x: fitness_function(vertices, distance_function, x), candidate_solutions))
        pheromones = update_pheromone(pheromones, candidate_solutions, fitness_values, Q=Q, rho=rho)
        
        for candidate_solution, fitness_value in zip(candidate_solutions, fitness_values):
            if fitness_value < best_fitness:
                best_fitness = fitness_value
                best_solution = candidate_solution
                
        log_of_best_distances.append(np.min(fitness_values))
        
        print(f"{i:8}:\t{np.min(fitness_values):5.8f}\t{np.mean(fitness_values):5.8f}\t{np.max(fitness_values):5.8f}")
    return best_solution, pheromones, log_of_best_distances

best_solution, pheromones, log_of_best_distances = ant_solver(vertices, distance_function)


Iteration	Minimum value	Mean value	Maximum value


TypeError: list indices must be integers or slices, not Request

In [ ]:


from matplotlib import pyplot as plt


plt.plot(log_of_best_distances)
plt.ylabel("Distance of the best path found")
plt.xlabel("Iteration")
plt.show()

In [ ]:
# Render pheromones (blue, line width corresponds to the pheromon value on the edge)
lines = []
colors = []
for i, v1 in enumerate(vertices):
    for j, v2 in enumerate(vertices):
        lines.append([(v1.x, v1.y), (v2.x, v2.y)])
        colors.append(pheromones[i, j])

lc = mc.LineCollection(lines, linewidths=np.array(colors))

plt.figure(figsize=(12, 8))
ax = plt.gca()
ax.add_collection(lc)
ax.autoscale()

solution = best_solution

# Here, you can test the influence of the individual parameters on the generated candidate solution
# solution = list(generate_solutions(vertices, pheromones, distance_function, number_of_ants=1, alpha=3, beta=1))[0]

# Print solution's fitness
print("Fitness: ", fitness_function(vertices, distance_function, solution))

# Render the solution (red)
solution_lines = []
for i, j in zip(solution, solution[1:] + solution[0:1]):
    solution_lines.append([(vertices[i].x, vertices[i].y), (vertices[j].x, vertices[j].y)])

solutions_lc = mc.LineCollection(solution_lines, colors="red")

ax.add_collection(solutions_lc)

plt.show()

# Print towns in the order of the soultion
solution_vertices = [vertices[i] for i in solution]
pprint.pprint(solution_vertices)